In [123]:
import pandas as pd #for adding dataframe support
import numpy as np
from sklearn.model_selection import train_test_split #for splitting the data set into training and testing data
from sklearn.feature_extraction.text import TfidfVectorizer #for vectorization of dataset
from sklearn.linear_model import LogisticRegression #prediction model used for this dataset which contains categorical data
from sklearn.metrics import accuracy_score, classification_report #for measuring statistics for the data
import nltk #for downloading necessary resources
from nltk.corpus import stopwords #including stopwords array
import string # for string manipulation
from textblob import Word # used later for lemmatization

# Download necessary NLTK resources (run this once)
nltk.download('stopwords') # common words that often don't hold semantic meaning\
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rendeeru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [124]:
drugs = pd.read_csv('drugLibTrain_raw.tsv', sep = '\t') #pd.read_csv is used to load the dataframe however since the datta is in .tsv format, sep= '\t'
                                                        #is used because the values of a .tsv file is separated by tabs


In [125]:
clean_drugs = drugs.dropna(subset=['sideEffectsReview']) #remove rows with null values in the sideEffectsReview column

In [126]:
def splitstring(x): #function for lowercasing strings
    x = " ".join(x.lower() for x in x.split()) #line used to lowercase string
    return x #return lowercased string

clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].apply(splitstring) #applies the lowercasing function to all strings in the column

clean_drugs.head(5) #view first 5 rows from the dataset

/tmp/ipykernel_1543/468633167.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].apply(splitstring) #applies the lowercasing function to all strings in the column


,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"heavy cycle, cramps, hot flashes, fatigue, lon...","I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,i felt extremely drugged and dopey. could not ...,See above


In [127]:
clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].str.replace('[^\w\s]', "", regex = True) #removes punctuation from string

clean_drugs.head(5) #view first 5 rows from the dataset

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_1543/3553593620.py:1: SyntaxWarning: invalid escape sequence '\w'
  clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].str.replace('[^\w\s]', "", regex = True) #removes punctuation from string
/tmp/ipykernel_1543/3553593620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].str.replace('[^\w\s]', "", regex = True) #removes punctuation from string


,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,cough hypotension proteinuria impotence rena...,"monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,heavy cycle cramps hot flashes fatigue long la...,"I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,heavier bleeding and clotting than normal,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,constipation dry mouth and some mild dizziness...,I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,i felt extremely drugged and dopey could not d...,See above


In [128]:
stop = stopwords.words('english') #initialize stopwords array
def replace_stopwords(x): #function for removing stopword
    x = " ".join(x for x in x.split() if x not in stop) #removes stopwords from single string
    return x #return cleaned string

    
clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].apply(replace_stopwords)  #applies the stopwords removing function to all strings in the column


/tmp/ipykernel_1543/1105320111.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].apply(replace_stopwords)  #applies the stopwords removing function to all strings in the column


In [129]:
clean_drugs #view dataset

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,cough hypotension proteinuria impotence renal ...,"monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,heavy cycle cramps hot flashes fatigue long la...,"I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,heavier bleeding clotting normal,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,constipation dry mouth mild dizziness would go...,I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,felt extremely drugged dopey could drive med a...,See above
...,...,...,...,...,...,...,...,...,...
3102,1039,vyvanse,10,Highly Effective,Mild Side Effects,adhd,"Increased focus, attention, productivity. Bett...",restless legs night insomnia headache sometime...,"I took adderall once as a child, and it made m..."
3103,3281,zoloft,1,Ineffective,Extremely Severe Side Effects,depression,Emotions were somewhat blunted. Less moodiness.,weight gain extreme tiredness day insomnia nig...,I was on Zoloft for about 2 years total. I am ...
3104,1664,climara,2,Marginally Effective,Moderate Side Effects,total hysterctomy,---,constant issues patch staying called manufactu...,---
3105,2621,trileptal,8,Considerably Effective,Mild Side Effects,epilepsy,Controlled complex partial seizures.,dizziness fatigue nausea,Started at 2 doses of 300 mg a day and worked ...


In [130]:
def lemmatizedrugs(x): #function for lemmatization
    x = " ".join([Word(word).lemmatize() for word in x.split()]) #lemmatizes or replaces variations of words with its base word
    return x #return lemmatized string

clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].apply(lemmatizedrugs) #lemmatizes the column

/tmp/ipykernel_1543/2316159903.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_drugs['sideEffectsReview'] = clean_drugs['sideEffectsReview'].apply(lemmatizedrugs) #lemmatizes the column


In [131]:
clean_drugs #view dataset

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,cough hypotension proteinuria impotence renal ...,"monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,heavy cycle cramp hot flash fatigue long lasti...,"I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,heavier bleeding clotting normal,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,constipation dry mouth mild dizziness would go...,I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,felt extremely drugged dopey could drive med a...,See above
...,...,...,...,...,...,...,...,...,...
3102,1039,vyvanse,10,Highly Effective,Mild Side Effects,adhd,"Increased focus, attention, productivity. Bett...",restless leg night insomnia headache sometimes...,"I took adderall once as a child, and it made m..."
3103,3281,zoloft,1,Ineffective,Extremely Severe Side Effects,depression,Emotions were somewhat blunted. Less moodiness.,weight gain extreme tiredness day insomnia nig...,I was on Zoloft for about 2 years total. I am ...
3104,1664,climara,2,Marginally Effective,Moderate Side Effects,total hysterctomy,---,constant issue patch staying called manufactur...,---
3105,2621,trileptal,8,Considerably Effective,Mild Side Effects,epilepsy,Controlled complex partial seizures.,dizziness fatigue nausea,Started at 2 doses of 300 mg a day and worked ...


In [134]:
X = clean_drugs['sideEffectsReview'] #puts the reviews on the x axis
y = clean_drugs['sideEffects'] #puts the corresponding side effects category on the y axis

X_train, X_test, y_train, y_test = train_test_split( #splits the data into training and testing data
    X, y, test_size = 0.2, random_state = 20, stratify = y) #parameters for splitting, the 2 colums, 20% for the testing data, some random state, and we stratify

In [135]:
X_train.head(10) #view the first 10 training data entries

2148    side effect far slight itch around injectionsi...
1098    sever irritation peeling outerlayer skin redne...
1425    felt like hopped whole time skin would feel cr...
660     ive taken lamictal different combo anticonvuls...
932     increase bloating increased feeling needing ur...
1831     unpleasant side effect estradiol could ascertain
1416                                 side effect remember
55      taking medicaion experience side effect thing ...
2228    initially frist two week medication patient co...
1125    erectile problemssevere anxiety panic attack n...
Name: sideEffectsReview, dtype: object

In [137]:
vectorizer = TfidfVectorizer( #use the TfidVectorizer function to create a vectorizer object
    lowercase=True, stop_words='english', max_df=0.95, min_df=2, ngram_range=(1, 2)) #parameters for the vectorizer
#min and max df allows us to ignore words that appear too often or too infrequently

X_train_tfidf = vectorizer.fit_transform(X_train)  # calculates the statistics needed, then tranforms the data into numerical value from the statistics for the model to use

X_test_tfidf = vectorizer.transform(X_test) # transforms the model, it doesn't need to fit because the statistics are still in the vectorizer

In [138]:
model = LogisticRegression(random_state=42, solver='liblinear')  #Using logistic regression because the we wish to analyze categorical data

model.fit(X_train_tfidf, y_train) # training the model

y_pred = model.predict(X_test_tfidf) #make predictions using the test data

accuracy = accuracy_score(y_test, y_pred) # evaluate the model on its accuracy
report = classification_report(y_test, y_pred) # create a report of more details abo

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

Accuracy: 0.5305
Classification Report:
                                precision    recall  f1-score   support

Extremely Severe Side Effects       0.67      0.06      0.11        35
            Mild Side Effects       0.50      0.76      0.60       203
        Moderate Side Effects       0.36      0.23      0.28       123
              No Side Effects       0.68      0.75      0.71       172
          Severe Side Effects       0.32      0.12      0.18        74

                     accuracy                           0.53       607
                    macro avg       0.51      0.38      0.38       607
                 weighted avg       0.51      0.53      0.49       607



In [139]:
print(clean_drugs['sideEffects'].value_counts())

sideEffects
Mild Side Effects                1016
No Side Effects                   858
Moderate Side Effects             614
Severe Side Effects               369
Extremely Severe Side Effects     175
Name: count, dtype: int64
